### Quick check for linkrot in the harvested endpoints, both sets

using a head request to start

With timeouts and capturing redirects


In [ ]:
import glob
import json
import os
import requests
from datetime import datetime

import logging
reload(logging)

logger = logging.getLogger(__name__)
handler = logging.FileHandler(filename="harvest_linkrot_%s.log" % datetime.now().strftime('%Y%m%d-%H%M'), mode="a", encoding="UTF-8")
logger.addHandler(handler)
logger.setLevel(logging.INFO)

files = glob.glob('../solr_superset/docs/*.json')

for i, f in enumerate(files):
    if i % 5000 == 0 and i > 0:
        print '{0} complete'.format(i)
    
    with open(f, 'r') as g:
        data = json.loads(g.read())
    
    url = data['url']
    
    try:
        req = requests.head(url, allow_redirects=True, timeout=10)
    except Exception as ex:
        logger.error('ERROR :: {0} :: {1} :: '.format(url, ex))
        continue
        
    if req.url != url:
        logger.info('REDIRECT :: {0} :: {1} :: {2}'.format(url, req.status_code, req.url))
    
    logger.info('RESPONSE :: {0} :: {1} :: '.format(url, req.status_code))
